In [1]:
#! pip install ipywidgets

In [12]:
from ipywidgets import interact, fixed
from ipywidgets.widgets import FloatRangeSlider, IntRangeSlider

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import binomial_options_pricing_model.bopm as bopm

In [13]:
risk_free_rate_slider = FloatRangeSlider(value=(0.01, 0.25), step=0.01, min=0.01, max=0.25, description='Risk-free rate:')
spot_price_slider = FloatRangeSlider(value=(40, 60), step=0.5, min=40, max=60, description='Spot price:')
strike_slider = FloatRangeSlider(value=(40, 60), step=0.5, min=40, max=60, description='Strike price:')
sigma_slider = FloatRangeSlider(value=(0.1, 0.5), step=0.05, min=0.01, max=0.5, description='Volatility:')
maturity_slider = IntRangeSlider(value=(1, 36), min=1, max=36, description='Maturity (in months):')

In [14]:
r, S, K, delta_t, T, sigma = .02, 50, 48, 1/12, 2, .3

def v_r(r, american = False, call = True):
    n = len(r)
    V = np.zeros(n)
    for i in range(n):
        V[i], _ = bopm.crr_price_option(r[i], S, K, delta_t, T, sigma, american, call)   
    return V

def v_s(S, american = False, call = True):
    n = len(S)
    V = np.zeros(n)
    for i in range(n):
        V[i], _ = bopm.crr_price_option(r, S[i], K, delta_t, T, sigma, american, call)   
    return V

def v_k(K, american = False, call = True):
    n = len(K)
    V = np.zeros(n)
    for i in range(n):
        V[i], _ = bopm.crr_price_option(r, S, K[i], delta_t, T, sigma, american, call)   
    return V

def v_sigma(sigma, american = False, call = True):
    n = len(sigma)
    V = np.zeros(n)
    for i in range(n):
        V[i], _ = bopm.crr_price_option(r, S, K, delta_t, T, sigma[i], american, call)   
    return V

def v_t(T, american = False, call = True):
    n = len(T)
    V = np.zeros(n)
    for i in range(n):
        V[i], _ = bopm.crr_price_option(r, S, K, delta_t, T[i], sigma, american, call)   
    return V

In [16]:
def plot_crr(v_func, param_slider, x_lab, is_time_param = False):
    if not is_time_param:
        param = np.linspace(*param_slider)
    else:
        param = (np.arange(*param_slider)+1)/12
        
    V_eu_call = v_func(param, american=False, call=True)
    V_eu_put = v_func(param, american=False, call=False)
    V_am_call = v_func(param, american=True, call=True)
    V_am_put = v_func(param, american=True, call=False)
    
    plt.plot(param, V_eu_call, color="blue", linestyle="dashed")
    plt.plot(param, V_eu_put, color="blue", linestyle="dashed")
    plt.plot(param, V_am_call, color="red")
    plt.plot(param, V_am_put, color="red")
    
    plt.legend(["EU call", "EU put", "AM call", "AM put"])
    plt.xlabel(x_lab)
    plt.ylabel("Option value V")
    plt.show()

interact(plot_crr, v_func=fixed(v_r), param_slider=risk_free_rate_slider, x_lab=fixed("Risk-free rate r"), is_time_param=fixed(False))
interact(plot_crr, v_func=fixed(v_s), param_slider=spot_price_slider, x_lab=fixed("Spot price $S_0$"), is_time_param=fixed(False))
interact(plot_crr, v_func=fixed(v_k), param_slider=strike_slider, x_lab=fixed("Strike price K"), is_time_param=fixed(False))
interact(plot_crr, v_func=fixed(v_sigma), param_slider=sigma_slider, x_lab=fixed("Volatility $\sigma$"), is_time_param=fixed(False))
interact(plot_crr, v_func=fixed(v_t), param_slider=maturity_slider, x_lab=fixed("Maturity T (in months)"), is_time_param=fixed(True))

interactive(children=(FloatRangeSlider(value=(0.01, 0.25), description='Risk-free rate:', max=0.25, min=0.01, …

interactive(children=(FloatRangeSlider(value=(40.0, 60.0), description='Spot price:', max=60.0, min=40.0, step…

interactive(children=(FloatRangeSlider(value=(40.0, 60.0), description='Strike price:', max=60.0, min=40.0, st…

interactive(children=(FloatRangeSlider(value=(0.11, 0.5), description='Volatility:', max=0.5, min=0.01, step=0…

interactive(children=(IntRangeSlider(value=(1, 36), description='Maturity (in months):', max=36, min=1), Outpu…

<function __main__.plot_crr(v_func, param_slider, x_lab, is_time_param=False)>